In [63]:
import pandas as pd
import json
import os
dir = os.getcwd()
# Read the xlsx file
df = pd.read_excel(dir+'\\transactionByDays.xlsx')

In [64]:
df.rename(columns={'日期':'Date', "住宅签约数量":"amount"}, inplace=True)

In [54]:
# parse the date
df['Date'] = pd.to_datetime(df['Date'])

In [65]:
dates = df['Date']
amounts = df['amount']

In [67]:
rawdatajson = {
    'dates': dates.to_list(),
    'amounts': amounts.to_list()
}

rawdatajson
with open(dir+'\\transaction-days.json', 'w') as f:
    json.dump(rawdatajson, f)

数据格式如下：
Date	amount
0	2023-11-11 00:00:00+00:00	21
1	2023-11-10 00:00:00+00:00	131
2	2023-11-09 00:00:00+00:00	147
3	2023-11-08 00:00:00+00:00	140

需要的json格式如下：
{
     2012: {"amount":100,
        "Jan":{"amount":100,
            ...
        },
        "Feb":{"amount":100,
            ...
        },
        ...
     },
     2013:{"amount":100,
        ...
     }
}

In [41]:
# 数据格式如下：
# Date	amount
# 0	2023-11-11 00:00:00+00:00	21
# 1	2023-11-10 00:00:00+00:00	131
# 2	2023-11-09 00:00:00+00:00	147
# 3	2023-11-08 00:00:00+00:00	140

# 需要的json格式如下：
# {
#      2012: {"amount":100,
#         "Jan":{"amount":100,
#             ...
#         },
#         "Feb":{"amount":100,
#             ...
#         },
#         ...
#      },
#      2013:{"amount":100,
#         ...
#      }
# }

# 1. 按年分组
df['year'] = df['Date'].dt.year

# 2. 按月分组
df['month'] = df['Date'].dt.month

# 3. 按日分组
df['day'] = df['Date'].dt.day




In [42]:
df

,Date,amount,year,month,day
0,2023-11-11 00:00:00+00:00,21,2023,11,11
1,2023-11-10 00:00:00+00:00,131,2023,11,10
2,2023-11-09 00:00:00+00:00,147,2023,11,9
3,2023-11-08 00:00:00+00:00,140,2023,11,8
4,2023-11-07 00:00:00+00:00,109,2023,11,7
...,...,...,...,...,...
2885,2015-09-08 00:00:00+00:00,414,2015,9,8
2886,2015-09-07 00:00:00+00:00,407,2015,9,7
2887,2015-09-06 00:00:00+00:00,388,2015,9,6
2888,2015-09-05 00:00:00+00:00,0,2015,9,5


In [47]:
# 转换成json, 上一级的amount是这一级的所有amount的总和

datajson = {}
for index, row in df.iterrows():
    year = row['year']
    month = row['month']
    day = row['day']
    amount = row['amount']
    if year not in datajson:
        datajson[year] = {"amount":amount}
    else:
        datajson[year]["amount"] += amount
    if month not in datajson[year]:
        datajson[year][month] = {"amount":amount}
    else:
        datajson[year][month]["amount"] += amount
    if day not in datajson[year][month]:
        datajson[year][month][day] = {"amount":amount}
    else:
        datajson[year][month][day]["amount"] += amount

datajson


{2023: {'amount': 25997,
  11: {'amount': 1113,
   11: {'amount': 21},
   10: {'amount': 131},
   9: {'amount': 147},
   8: {'amount': 140},
   7: {'amount': 109},
   6: {'amount': 120},
   5: {'amount': 18},
   4: {'amount': 32},
   3: {'amount': 122},
   2: {'amount': 151},
   1: {'amount': 122}},
  10: {'amount': 2802,
   31: {'amount': 115},
   30: {'amount': 134},
   29: {'amount': 15},
   28: {'amount': 16},
   27: {'amount': 142},
   26: {'amount': 149},
   25: {'amount': 150},
   24: {'amount': 174},
   23: {'amount': 149},
   22: {'amount': 18},
   21: {'amount': 10},
   20: {'amount': 141},
   19: {'amount': 144},
   18: {'amount': 179},
   17: {'amount': 138},
   16: {'amount': 138},
   15: {'amount': 20},
   14: {'amount': 25},
   13: {'amount': 150},
   12: {'amount': 137},
   11: {'amount': 169},
   10: {'amount': 120},
   9: {'amount': 129},
   8: {'amount': 116},
   7: {'amount': 104},
   6: {'amount': 10},
   5: {'amount': 7},
   4: {'amount': 1},
   3: {'amount': 1},


In [48]:
# save the json file
with open(dir+'\\transactionByDays.json', 'w') as f:
    json.dump(datajson, f)